In [7]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
import numpy as np

In [8]:
model = read_sbml_model('iMM904.xml') 

In [9]:
# Create stoichiometric matrix
S = create_stoichiometric_matrix(model)
deg_of_freedom = S.shape[1] - np.linalg.matrix_rank(S)

In [10]:
# S_bin should only contain floating point numbers (either 0. or 1.)
S_bin = np.where(S != 0, 1, 0)
# Compute reaction adjacency matrix A_v.
A_v = S_bin.T.dot(S_bin)
# Compute compound adjacency matrix A_x.
A_x = S_bin.dot(S_bin.T)

In [11]:
# Based on A_x, which metabolite participates in the most reactions?
most_connected_metabolite = sorted(zip(model.metabolites, A_x.diagonal()), key=lambda item: item[1], reverse=True)[0][0]

In [13]:
model.metabolites[0:10] #The model contains a list of metabolites. Here are the first ten.

[<Metabolite 2dr5p_c at 0x1e367dd3f60>,
 <Metabolite 2hb_c at 0x1e367dd35c0>,
 <Metabolite 2hb_e at 0x1e367dd3ac8>,
 <Metabolite 2hhxdal_c at 0x1e367dd3dd8>,
 <Metabolite 2hp6mbq_m at 0x1e367dd3160>,
 <Metabolite 2hp6mp_m at 0x1e367dd30b8>,
 <Metabolite 2hpmhmbq_m at 0x1e367dc3f60>,
 <Metabolite 2hpmmbq_m at 0x1e367dc3b70>,
 <Metabolite 2ippm_c at 0x1e367dc32b0>,
 <Metabolite 2kmb_c at 0x1e367dc3390>]

In [14]:
model.optimize()

,fluxes,reduced_costs
CITtcp,0.000000,0.000000e+00
13BGH,0.000000,-4.751566e-02
13BGHe,0.000000,0.000000e+00
13GS,0.326670,0.000000e+00
16GS,0.000000,0.000000e+00
...,...,...
PYDXO,0.000000,0.000000e+00
PYK,17.721990,-1.387779e-17
PYNP2r,0.000000,0.000000e+00
PYR5CDm,0.000000,-2.969729e-02


In [15]:
model.optimize().objective_value

0.28786570370401743